In [1]:
include("saltsolver.jl")
include("passive.jl")

smallest_stability_eigs (generic function with 1 method)

In [6]:
println("Cnv, uniform everything")

N = 3
nsym = N
Lcav = 1.0
ℓ = 1
σ = 0.1
γ⟂ = 0.2
ωℓ = N * √(2*(1-cos(2π*ℓ/N)))
ωa = 4.0
D = 0.2
ɛ = (1+im*σ) * ones(N)
F = ones(N)
las = Laser(ɛ, F, ωa, γ⟂)

L = 1.0
h = L/N
Ering, ωring = ring_eigenpair(N, ℓ, h)
md = PassiveMode(copy(Ering), ωring)
laplacian!(J) = periodic!(J, h)
Dt = passive_threshold!(laplacian!, md, las, (1.0, 1.2))
ωt = real(md.ω)
Eplus = md.E / md.E[1]
Es = (Eplus, ring_flip(Eplus))

H, Is, Js, Ks, GD = overlap_integrals(Es, ωt, Dt, las, nsym, ℓ, Lcav);
I, J, K = Is[1], Js[1], Ks[1];
i, j, k = I/H, J/H, K/H;

Dt, imag(md.ω)

Cnv, uniform everything


(0.9332690479444662,9.3237349405047e-11)

In [14]:
real(H) - (2ωℓ^2/ωt^3 - σ/γ⟂)*(ωt-ωa)/γ⟂,
imag(H) - (2ωℓ^2/ωt^3 + σ/γ⟂)

(-8.706695336924142e-9,-3.93424381961438e-9)

In [5]:
α = (γ⟂/σ - ωa)/3
β = α^3 - γ⟂*ωring^2/(2σ)
v = (-β + √(β^2-α^6))^(1/3)
ω = v + α^2/v - α
ωring^2 - ω^2*(1 + σ/γ⟂*(ω-ωa))

1.4210854715202004e-14

In [12]:
ℓ = 3
npiece = 4
Lcav = 1.0
wantcnv = true

nsym = 4ℓ
N = nsym * npiece
h = Lcav/N
Ering, ωring = ring_eigenpair(N, ℓ, h)
γ⟂ = 0.8
ωa = round(ωring)

ctrand(n) = rand(n)-0.5
ɛrpiece = 1 + 0.3*ctrand(npiece)
ɛipiece = 1.5 * rand(npiece) # want positive
Fpiece = 1 + 0.4*ctrand(npiece)

cnvize!(v) = (v[:] = (v + ring_flip(v))/2)
if wantcnv
    cnvize!(ɛrpiece)
    cnvize!(ɛipiece)
    cnvize!(Fpiece)
end

ɛ = cn_profile(ɛrpiece + im*ɛipiece, nsym)
F = cn_profile(Fpiece, nsym)
las = Laser(ɛ, F, ωa, γ⟂)
laplacian!(J) = periodic!(J, h)

md = Mode(copy(Ering), ωring, 0.0)
Dguesses = (0.6, 0.8)
Dt = threshold!(laplacian!, md, las, Dguesses)
ωt = md.ω
Es = (copy(md.E), ring_flip(md.E))

H, Is, Js, Ks, GD = overlap_integrals(Es, ωt, Dt, las, nsym, ℓ, Lcav);
I, J, K = Is[1], Js[1], Ks[1];
i, j, k = I/H, J/H, K/H;

println("H = ", H, '\n')

# for z² in (1, -1)
# #     println(2imag(2i+j + z²*k))
#     radicand = imag(j - z²*k)^2 + 8real(k)*real(j + z²*k)
#     term = -imag(j + 3z²*k)
#     println("term = ", term)
#     println("√rad = ", √radicand)
#     for sn in (1, -1)
#         println(term + sn*√radicand)
#     end
#     z² == 1 && println()
# end

H = 0.08661900749283337 + 1.1160234858646176im

